In [1]:
! wget https://raw.githubusercontent.com/google/sentencepiece/master/src/sentencepiece_model.proto
! protoc --python_out=. sentencepiece_model.proto

wget: /data/yash/miniconda3/lib/libuuid.so.1: no version information available (required by wget)
--2022-05-22 13:09:34--  https://raw.githubusercontent.com/google/sentencepiece/master/src/sentencepiece_model.proto
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12872 (13K) [text/plain]
Saving to: ‘sentencepiece_model.proto.1’

sentencepiece_model 100%[===================>]  12.57K  --.-KB/s    in 0.003s  

2022-05-22 13:09:35 (3.77 MB/s) - ‘sentencepiece_model.proto.1’ saved [12872/12872]



In [2]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer
tokenizer = T5Tokenizer.from_pretrained("google/mt5-base")
model = T5ForConditionalGeneration.from_pretrained('google/mt5-base')

You are using a model of type mt5 to instantiate a model of type t5. This is not supported for all configurations of models and can yield errors.


In [3]:
def msize(m):
    return sum(p.numel() for p in m.parameters())
print(msize(model.shared) / msize(model))   # 0.3298
print(msize(model.lm_head) / msize(model))  # 0.3298

0.32981729710484153
0.32981729710484153


In [4]:
import pandas as pd
import csv
import tqdm
from collections import Counter
from tqdm.auto import tqdm, trange
data =[]
with open('en-hi/train.hi','r') as f:
    data = [s.strip('\n').replace('  ', ' ') for s in f.readlines()]
cnt_ru = Counter()
for text in tqdm(data):
    cnt_ru.update(tokenizer.encode(text))
print(len(cnt_ru), len(cnt_ru)/tokenizer.vocab_size)

  0%|          | 0/8466307 [00:00<?, ?it/s]

67652 0.270499800079968


In [5]:
data =[]
with open('en-hi/train.en','r') as f:
    data = [s.strip('\n').replace('  ', ' ') for s in f.readlines()]
cnt_en = Counter()
for text in tqdm(data):
    cnt_en.update(tokenizer.encode(text))
print(len(cnt_en), len(cnt_en)/tokenizer.vocab_size)

  0%|          | 0/8466307 [00:00<?, ?it/s]

82029 0.32798480607756897


In [6]:
common = len(set(cnt_ru.keys()).intersection(set(cnt_en.keys())))

In [7]:
print(common, common / len(cnt_ru))

54771 0.8095991249334831


In [8]:
print('ru')
for top in 10_000, 20_000, 30_000:
    print(top, sum(v for k, v in cnt_ru.most_common(top)) / sum(cnt_ru.values()))
print('en')
for top in 10_000, 20_000, 30_000:
    print(top, sum(v for k, v in cnt_en.most_common(top)) / sum(cnt_en.values()))

ru
10000 0.9984905625596253
20000 0.9994138378817895
30000 0.9997237355070604
en
10000 0.9564890253740371
20000 0.9859295854825768
30000 0.9944200294217131


In [9]:
old_voc = tokenizer.get_vocab()
old_inv_voc = {v: k for k, v in old_voc.items()}

In [10]:
new_tokens = set(range(1000))
for i, (k, v) in enumerate(cnt_en.most_common(35_000)):
    if k not in new_tokens:
        new_tokens.add(k)
for i, (k, v) in enumerate(cnt_ru.most_common(40_000)):
    if len(new_tokens) == 80_900:
        print(i, 'Hindi tokens are included')
        break
    if k not in new_tokens:
        new_tokens.add(k)
for t in range(tokenizer.vocab_size - 100, tokenizer.vocab_size):
    new_tokens.add(t)
print(len(new_tokens))
kept_ids = sorted(new_tokens)

51511


In [11]:
new_size = len(kept_ids)
new_emb = torch.nn.Embedding(new_size, model.shared.embedding_dim)
new_head = torch.nn.Linear(in_features=model.lm_head.in_features, out_features=new_size, bias=False)

In [12]:
for new_id, old_id in enumerate(kept_ids):
    new_emb.weight.data[new_id] = model.shared.weight.data[old_id]
    new_head.weight.data[new_id] = model.lm_head.weight.data[old_id]

In [13]:
model.shared.weight = new_emb.weight
model.lm_head.weight = new_head.weight

In [14]:
import sentencepiece_model_pb2 as spmp
smp = tokenizer.sp_model.serialized_model_proto()
m = spmp.ModelProto()
m.ParseFromString(smp)

print('the loaded model has pieces:', len(m.pieces))
new_pieces = [m.pieces[idx] for idx in kept_ids]
print('the new pieces:', len(new_pieces))

# replace the content of the first 30K pieces
for i, p in enumerate(new_pieces):
    m.pieces[i].piece = p.piece
    m.pieces[i].score = p.score
    m.pieces[i].type = p.type

# drop the remaining pieces
n = len(new_pieces)
for i in trange(len(m.pieces) - n):
    m.pieces.pop(len(m.pieces) - 1)

print(len(m.pieces))
with open('new_sp.model', 'wb') as f:
    f.write(m.SerializeToString())

the loaded model has pieces: 250100
the new pieces: 51511


  0%|          | 0/198589 [00:00<?, ?it/s]

51511


In [15]:
new_tokenizer = T5Tokenizer('new_sp.model', extra_ids=0)

In [22]:
model.config.__dict__['vocab_size'] = new_size
model.config.__dict__['_name_or_path'] = 'cointegrated/hit5-base'
model.config

T5Config {
  "_name_or_path": "cointegrated/hit5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings": false,
  "tokenizer_class": "T5Tokenizer",
  "torch_dtype": "float32",
  "transformers_version": "4.19.2",
  "use_cache": true,
  "vocab_size": 51511
}

In [23]:
new_tokenizer.save_pretrained('hit5-base')
model.save_pretrained('hit5-base')

In [24]:
!ls hin5-base -alsh

total 1.1G
4.0K drwxrwxr-x  2 yash yash    4.0K May 22 14:05 .
4.0K drwxrwxr-x 31 yash prospar 4.0K May 22 14:15 ..
4.0K -rw-rw-r--  1 yash yash     748 May 22 14:05 config.json
1.1G -rw-rw-r--  1 yash yash    1.1G May 22 14:05 pytorch_model.bin
4.0K -rw-rw-r--  1 yash yash      65 May 22 14:05 special_tokens_map.json
1.1M -rw-rw-r--  1 yash yash    1.1M May 22 14:05 spiece.model
4.0K -rw-rw-r--  1 yash yash     173 May 22 14:05 tokenizer_config.json


In [25]:
model1 = T5ForConditionalGeneration.from_pretrained('hit5-base')
tokenizer1 = T5Tokenizer.from_pretrained('hit5-base')


In [1]:
from simpletransformers.t5 import T5Model, T5Args

In [3]:
import logging

In [4]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [5]:
model_args = T5Args()
model_args.max_seq_length = 64
model_args.train_batch_size = 8
model_args.eval_batch_size = 8
model_args.num_train_epochs = 4
model_args.evaluate_during_training_steps = 30000
model_args.use_multiprocessing = False
model_args.fp16 = False
model_args.save_steps = -1
model_args.save_eval_checkpoints = False
model_args.no_cache = True
model_args.reprocess_input_data = True
model_args.overwrite_output_dir = True
model_args.preprocess_inputs = False
model_args.num_beams = 3
model_args.num_return_sequences = 1


In [10]:
model = T5Model('mt5','hit5-base', args=model_args)

You are using a model of type t5 to instantiate a model of type mt5. This is not supported for all configurations of models and can yield errors.


In [11]:
model.config

MT5Config {
  "_name_or_path": "hit5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "mt5",
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings": false,
  "tokenizer_class": "T5Tokenizer",
  "torch_dtype": "float32",
  "transformers_version": "4.19.2",
  "use_cache": true,
  "vocab_size": 51511
}